<a href="https://colab.research.google.com/github/antoinebachand/Deep-Learnig-/blob/main/Neural_thermo_2target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network for Prediction of Hydrogen Thermodynamics**
### Author: Antoine Bachand (antoinebachand@outlook.com)

The training data set is obtained through a model adapted from Kushnir et al. (2012) for hydrogen storage. DOI: 10.1016/j.ijheatmasstransfer.2012.05.055 

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as functional
import torch.nn as nn

# Loading data

In [10]:
#load the data
df = pd.read_csv('data.csv')
df.columns = ['Injection Temperature (K)', 'Intitial Temperature (K)', 'Total Mass (kg)', 'Injection Time (s)', 'Max Pressure (MPa)','Max Temperature (Celsius)']


The inputs to the neural network are:
- Initial rock temperature (K)
- Injection temperature (K)
- Total injection mass (kg)
- Injection time (s) 

and the outputs are as follows:
- Maximum pressure ( MPa )
- Maximum temperature ( °C )  

In [11]:
df.head()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Max Pressure (MPa),Max Temperature (Celsius)
0,375.030504,309.254573,1008.433517,7802.912741,48.811092,91.867495
1,351.170951,275.184665,1443.385908,5630.677776,77.734770,87.824516
2,472.512070,305.464103,1251.924370,12234.423440,68.366702,112.732767
3,300.509319,288.600197,1228.477505,21488.955110,53.333108,36.905345
4,415.714802,297.733495,1891.034337,21949.876650,122.041692,96.263199


Since our different features are on different ranges, a normalization is applied. 

In [12]:
# Normalization
norm = MinMaxScaler().fit(df) 
nd_norm = norm.transform(df)
df_norm = pd.DataFrame(nd_norm)

df_norm.columns = ['Injection Temperature (K)', 'Intitial Temperature (K)', 'Total Mass (kg)', 'Injection Time (s)', 'Max Pressure (MPa)','Max Temperature (Celsius)']


In [13]:
df_norm.describe()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Max Pressure (MPa),Max Temperature (Celsius)
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.503607,0.520212,0.501635,0.511772,0.036351,0.016290
std,0.291841,0.291506,0.292448,0.279332,0.054182,0.048852
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.254583,0.270495,0.243196,0.285297,0.012749,0.007859
50%,0.505140,0.534312,0.501083,0.520112,0.028623,0.011561
75%,0.760425,0.788682,0.769284,0.741848,0.051027,0.015920
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#Preparing data for training/testing

Here we split the data between training (80%) and testing (20%) in order to test the neural network with unseen data

In [42]:
# The columns are split between independent (Input) and dependent (Output)
columns = df_norm.columns.values.tolist()
# The independent ones correspond to the last two columns of the data set. 
independent = columns[:-2]
dependent = columns[-2]                 # columns[-2:]

# The test data size is set at 20%. 
X_train, X_test, y_train, y_test = train_test_split(df_norm[independent],df_norm[dependent], test_size=0.2)


Now we will create the tensors. They correspond to the structure used to store information/data in machine learning. 

Also, since the stochastic gradient descent computes the gradient on a subset of the training data, as opposed to the entire training dataset, we need to specify a batch size. Therefore, to avoid generalization, we will use a small batch size of 50. 

In [43]:
BATCH_SIZE = 50
# Tensor for the training data and the value type is float 
X = torch.tensor(X_train.values, dtype=torch.float)
y = torch.tensor(y_train.values, dtype=torch.float)

train_ds = torch.utils.data.TensorDataset(X,y)

# Tensor for the testing data and the value type is float
X = torch.tensor(X_test.values, dtype=torch.float)
y = torch.tensor(y_test.values, dtype=torch.float)

test_ds = torch.utils.data.TensorDataset(X,y)

# We shuffle the training dataset
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# Network architecture

# Training

# Regularization